In [1]:
%%capture
!pip install pygame
!pip install pygad

!git clone https://github.com/karinemiras/evoman_framework.git tmp
!cp -r /kaggle/working/tmp/* /kaggle/working/
!rm -R /kaggle/working/tmp

In [2]:
import sys, os
import numpy as np
import pandas as pd

import random

from evoman.environment import Environment
from evoman.controller import Controller
from demo_controller import player_controller
from deap import base, creator, tools, algorithms
import multiprocessing
import optuna

import seaborn as sns
import matplotlib.pyplot as plt

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
def parameter_count(hidden_neurons):
    if hidden_neurons>0:
        n_w = (20*hidden_neurons) + (hidden_neurons*5)
        n_b = hidden_neurons + 5
        return n_w + n_b
    else:
        return (20*5)+5
    
def fitness_function(args):
        solution, train_enemies = args
        sum_gain = 0
        for enemy in train_enemies:
            env.update_parameter("enemies",[enemy])
            res = env.play(np.array(solution))
            gain = res[1]-res[2]
            sum_gain += gain
        return (sum_gain/len(train_enemies),)

"""
def fitness_function(solution):
    defeated = 0
    for enemy in range(1,9):
        env.update_parameter("enemies",[enemy])
        res = env.play(np.array(solution))
        if res[2]<=0:
            defeated += 1
    return (defeated,)
"""

'\ndef fitness_function(solution):\n    defeated = 0\n    for enemy in range(1,9):\n        env.update_parameter("enemies",[enemy])\n        res = env.play(np.array(solution))\n        if res[2]<=0:\n            defeated += 1\n    return (defeated,)\n'

In [4]:
def cxWholeAritmetic(parent1,parent2,alpha):
    child1 = (alpha*np.array(parent1))+((1-alpha)*np.array(parent2))
    child2 = (alpha*np.array(parent2))+((1-alpha)*np.array(parent1))
    return creator.Individual(child1),creator.Individual(child2)

def cxBlend(parent1,parent2,alpha):
    gamma = (1+2*alpha)*np.random.uniform(size=(len(parent1,))) - alpha
    child1 = (gamma*np.array(parent1))+((1-gamma)*np.array(parent2))
    child2 = (gamma*np.array(parent2))+((1-gamma)*np.array(parent1))
    return creator.Individual(child1),creator.Individual(child2)

In [5]:
def mutGaussian(ind,sigma,indpb):
    mutation = [np.random.normal(0,sigma) if np.random.uniform()<indpb else 0 for i in range(len(ind))]
    return creator.Individual(np.array(ind) + np.array(mutation))

def mutGaussianAdaptive(ind,sigma,tau,indpb):
    new_sigma = sigma * np.exp(np.random.normal(0,tau,size=(len(ind),)))
    new_sigma[new_sigma<0.1] = 0.1
    mutation = [np.random.normal(0,s) if np.random.uniform()<indpb else 0 for s in new_sigma]
    return creator.Individual(np.array(ind) + np.array(mutation))

In [6]:
def genetic_algorithm(train_enemies,initial_range,pop_size,n_offspring,tournsize_parent,cxpb,cx_type,
                      alpha,mutpb,mut_type,sigma,indpb,tau,tournsize_survival,ngen,verbose):
    
    global individual_size,env,creator,stats

    
    toolbox = base.Toolbox()
    toolbox.register("weight_bin", np.random.uniform,-initial_range,initial_range)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.weight_bin, n=individual_size)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", fitness_function)

    pop = toolbox.population(n=pop_size)
    hof = {'best_solution':-np.inf,'max_value':-np.inf}

    with multiprocessing.Pool() as pool:
        # evaluate initial population
        for ind, fit in zip(pop, pool.map(toolbox.evaluate,zip(pop,[train_enemies for i in range(len(pop))]))):
            ind.fitness.values = fit

        # record best solution
        for ind in pop:
            if ind.fitness.values[0]>hof['max_value']:
                hof['best_solution'] = ind
                hof['max_value'] = ind.fitness.values[0]

        logbook = tools.Logbook()
        logbook.header = ['gen', 'nevals'] + (stats.fields if stats else []) + ['max_hof']
        record = stats.compile(pop) if stats else {}

        logbook.record(gen=0, nevals=len(pop), **record, max_hof=hof['max_value'])
        if verbose==1:
            print(logbook.stream)

        for g in range(ngen):
            # select parents
            parents = tools.selTournament(individuals=pop,k=n_offspring*pop_size,tournsize=tournsize_parent)

            # create offspring
            offspring = []
            for child1, child2 in zip(parents[::2], parents[1::2]):
                if np.random.uniform() < cxpb:
                    if cx_type=="whole_aritmetic":
                        if alpha=="random":
                            alpha=np.random.uniform()
                            child1,child2 = cxWholeAritmetic(child1, child2, alpha)
                        else:
                            child1,child2 = cxWholeAritmetic(child1, child2, alpha)
                    elif cx_type=="blend":
                        if alpha=="random":
                            alpha=np.random.uniform()
                            child1,child2 = cxBlend(child1, child2, alpha)
                        else:
                            child1,child2 = cxBlend(child1, child2, alpha)
                offspring.append(child1)
                offspring.append(child2)

            # apply mutation to offspring
            for i in range(len(offspring)):
                if np.random.uniform() < mutpb:
                    if mut_type=="normal":
                        if indpb=="random":
                            indpb=np.random.uniform()
                            offspring[i] = mutGaussian(offspring[i],sigma,indpb)
                        else:
                            offspring[i] = mutGaussian(offspring[i],sigma,indpb)
                    elif mut_type=="adaptive":
                        if indpb=="random":
                            indpb=np.random.uniform()
                            offspring[i] = mutGaussianAdaptive(offspring[i],sigma,indpb,tau)
                        else:
                            offspring[i] = mutGaussianAdaptive(offspring[i],sigma,indpb,tau)       

            # evaluate offspring
            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            for ind, fit in zip(invalid_ind, pool.map(toolbox.evaluate,zip(invalid_ind,[train_enemies for i in range(len(invalid_ind))]))):
                    ind.fitness.values = fit

            # record best solution
            for ind in offspring:
                if ind.fitness.values[0]>hof['max_value']:
                    hof['best_solution'] = ind
                    hof['max_value'] = ind.fitness.values[0]

            # select the next generation of the population
            pop = tools.selTournament(individuals=offspring,k=pop_size,tournsize=tournsize_survival)

            record = stats.compile(pop) if stats else {}
            logbook.record(gen=g+1, nevals=len(invalid_ind), **record, max_hof=hof['max_value'])
            if verbose==1:
                print(logbook.stream)
        pool.close()
        pool.join()

    return hof,logbook

In [7]:
hidden_neurons = 10
individual_size = parameter_count(hidden_neurons)

headless = True
if headless:
    os.environ["SDL_VIDEODRIVER"] = "dummy"

env = Environment(playermode="ai",
                  player_controller=player_controller(hidden_neurons),
                  speed="fastest",
                  enemymode="static",
                  level=2,
                  visuals=False,
                  logs="off")

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("Mean", np.mean)
stats.register("Max", np.max)
stats.register("Std", np.std)

ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5704:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM default


In [8]:
def objective(trial):
    
    params = {'train_enemies':trial.suggest_categorical("train_enemies",[[7,8],[2,5,6],[1,2,6,7]]),
              'initial_range':trial.suggest_int("initial_range", 1,3,step=1),
              'pop_size':trial.suggest_int("pop_size", 50,50,step=50),
              'n_offspring':trial.suggest_int("n_offspring", 1,5,step=1),
              'tournsize_parent':trial.suggest_int("tournsize_parent", 1,10,step=1),
              'cxpb':trial.suggest_float("cxpb", 0.1, 1., step=0.1),
              'cx_type':trial.suggest_categorical("cx_type",["whole_aritmetic","blend"]),
              'alpha':trial.suggest_categorical("alpha",np.arange(0.1,1.1,0.1).tolist()+["random"]),
              'mutpb':trial.suggest_float("mutpb", 0.1, 1., step=0.1),
              'mut_type':trial.suggest_categorical("mut_type",["normal","adaptive"]),
              'sigma':trial.suggest_float("sigma", 0.5, 3, step=0.5),
              'indpb':trial.suggest_categorical("indpb",np.arange(0.1,1.1,0.1).tolist()+["random"]),
              'tau':trial.suggest_float("tau", 0.1, 1., step=0.1),
              'tournsize_survival':trial.suggest_int("tournsize_survival", 1,10,step=1),
              'ngen':trial.suggest_int("ngen", 30,30,step=20),
              'verbose':trial.suggest_int("verbose", 1,1,step=1)}
    
    global hof_tuning
    iteration_logs = []
    for i in range(3):
        hof,logs = genetic_algorithm(**params)
        logs_pd = pd.DataFrame(logs)
        iteration_logs.append(logs_pd)
        hof_tuning.append(hof)
    
    return pd.concat(iteration_logs)['Max'].mean()


In [9]:
hof_tuning = []

study = optuna.create_study(study_name="GA", direction="maximize")

study.optimize(objective, n_trials=500)

[I 2023-10-14 23:54:46,915] A new study created in memory with name: GA
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [7, 8] which is of type list.
  warnings.warn(message)
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [2, 5, 6] which is of type list.
  warnings.warn(message)
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [1, 2, 6, 7] which is of type list.
  warnings.warn(message)


gen	nevals	Mean   	Max	Std   	max_hof
0  	50    	-87.808	31 	27.248	31     
1  	90    	10.534 	31 	19.2636	31     
2  	91    	32.142 	36.1	2.03618	36.1   
3  	91    	34.21  	36.1	7.43183	36.1   
4  	93    	36.1   	36.1	0      	36.1   
5  	95    	36.1   	36.1	0      	36.1   
6  	91    	36.1   	36.1	0      	39.1   
7  	91    	36.1   	36.1	0      	39.1   
8  	91    	36.1   	36.1	0      	39.1   
9  	94    	36.1   	36.1	0      	39.1   
10 	94    	36.1   	36.1	0      	39.1   
11 	90    	36.1   	36.1	0      	39.1   
12 	90    	37.172 	47.2	3.87949	47.2   
13 	96    	43.78  	48.4	2.95039	48.4   
14 	86    	46.938 	48.4	8.61642	48.4   
15 	94    	48.4   	48.4	7.10543e-15	48.4   
16 	97    	48.4   	48.4	7.10543e-15	48.4   
17 	98    	47.296 	48.4	6.27407    	48.4   
18 	89    	46.636 	48.4	8.61175    	48.4   
19 	97    	48.4   	48.4	7.10543e-15	48.4   
20 	93    	47.494 	48.4	3.77452    	48.4   
21 	93    	48.4   	48.4	7.10543e-15	48.4   
22 	95    	48.4   	48.4	7.10543e-15	48.4   
23 	94    	46

[I 2023-10-14 23:59:28,177] Trial 0 finished with value: 46.54731182795728 and parameters: {'train_enemies': [7, 8], 'initial_range': 2, 'pop_size': 50, 'n_offspring': 2, 'tournsize_parent': 9, 'cxpb': 0.8, 'cx_type': 'whole_aritmetic', 'alpha': 0.4, 'mutpb': 0.6, 'mut_type': 'normal', 'sigma': 3.0, 'indpb': 1.0, 'tau': 0.2, 'tournsize_survival': 10, 'ngen': 30, 'verbose': 1}. Best is trial 0 with value: 46.54731182795728.


30 	90    	58.21  	58.6	2.73       	58.6   
gen	nevals	Mean    	Max     	Std    	max_hof 
0  	50    	-77.7773	-13.4667	19.2709	-13.4667
1  	132   	-5.84667	23.9333 	21.0927	23.9333 
2  	129   	23.8213 	23.9333 	0.46446	24.3333 
3  	133   	23.9333 	23.9333 	0      	24.3333 
4  	115   	23.9333 	23.9333 	0      	24.3333 
5  	105   	23.9333 	23.9333 	0      	24.3333 
6  	116   	23.9333 	23.9333 	0      	24.3333 
7  	105   	24.0493 	29.7333 	0.812  	29.7333 
8  	142   	27.2973 	29.7333 	2.86264	34.8667 
9  	120   	29.7333 	29.7333 	3.55271e-15	34.8667 
10 	124   	30.2413 	38.2    	2.01072    	38.2    
11 	112   	37.8613 	38.2    	1.65912    	38.2    
12 	125   	38.2    	38.2    	7.10543e-15	38.2    
13 	113   	38.2    	38.2    	7.10543e-15	38.2    
14 	125   	38.2    	38.2    	7.10543e-15	38.2    
15 	114   	38.2    	38.2    	7.10543e-15	38.2    
16 	125   	38.2    	38.2    	7.10543e-15	38.2    
17 	133   	38.2    	38.2    	7.10543e-15	38.2    
18 	126   	38.2    	38.2    	7.10543e-15	38.2 

[I 2023-10-15 00:08:46,926] Trial 1 finished with value: 38.063799283154246 and parameters: {'train_enemies': [2, 5, 6], 'initial_range': 3, 'pop_size': 50, 'n_offspring': 4, 'tournsize_parent': 9, 'cxpb': 0.5, 'cx_type': 'blend', 'alpha': 0.5, 'mutpb': 0.2, 'mut_type': 'adaptive', 'sigma': 1.5, 'indpb': 0.9, 'tau': 0.6, 'tournsize_survival': 10, 'ngen': 30, 'verbose': 1}. Best is trial 0 with value: 46.54731182795728.


...

30 	58    	58.3  	58.3	0          	58.3   
gen	nevals	Mean   	Max   	Std    	max_hof
0  	50    	-93.697	-26.85	15.1681	-26.85 
1  	144   	-49.57 	-20   	16.7708	-20    
2  	141   	-25.066	-14.1 	12.1556	-14.1  
3  	141   	-20.112	-14.1 	8.48953	-14.1  
4  	140   	-18.475	-1.25 	10.2491	-1.25  
5  	142   	-18.394	-1.25 	10.5887	-1.25  
6  	135   	-13.333	-1.25 	10.6211	-1.25  
7  	138   	-8.957 	5.75  	12.1382	5.75   
8  	141   	-7.54  	5.75  	11.8984	5.75   
9  	134   	-1.47  	6.25  	13.5065	6.25   
10 	145   	-6.485 	6.25  	11.8671	6.25   
11 	139   	-3.381 	6.25  	12.5786	6.25   
12 	141   	-4.176 	6.25  	12.2529	6.25   
13 	140   	-1.905 	7.25  	11.9028	7.25   
14 	138   	-1.969 	7.75  	12.2995	7.75   
15 	133   	3.003  	9.25  	9.17098	9.25   
16 	139   	5.118  	10.25 	8.46234	10.25  
17 	134   	6.84   	10.25 	8.49673	10.25  
18 	139   	4.928  	11.75 	10.5256	11.75  
19 	139   	2.27   	11.75 	12.0851	11.75  
20 	138   	0.212  	11.75 	14.4672	11.75  
21 	137   	2.475  	11.75 	14.8171

[I 2023-10-15 10:41:22,851] Trial 75 finished with value: 6.3494623655914335 and parameters: {'train_enemies': [1, 2, 6, 7], 'initial_range': 2, 'pop_size': 50, 'n_offspring': 3, 'tournsize_parent': 7, 'cxpb': 0.2, 'cx_type': 'whole_aritmetic', 'alpha': 0.7000000000000001, 'mutpb': 0.9, 'mut_type': 'normal', 'sigma': 2.0, 'indpb': 0.8, 'tau': 0.8, 'tournsize_survival': 10, 'ngen': 30, 'verbose': 1}. Best is trial 34 with value: 59.495698924731414.


30 	138   	0.682  	12.55	15.1302	12.55  
gen	nevals	Mean   	Max	Std    	max_hof
0  	50    	-89.914	4.3	22.3572	4.3    
1  	200   	-10.846	17.4	23.2386	17.4   
2  	200   	5.112  	36.4	14.9725	36.4   
3  	200   	19.812 	51.1	15.9613	51.1   
4  	200   	30.054 	69.4	18.4128	69.4   
5  	200   	26.026 	54.1	22.2443	69.4   
6  	200   	32.672 	54.1	16.1801	69.4   
7  	200   	47.556 	58  	9.73029	69.4   
8  	200   	33.886 	57.1	18.0292	69.4   
9  	200   	45.484 	61.3	15.5307	69.4   
10 	200   	33.034 	67.6	20.5292	69.4   
11 	200   	39.936 	61  	16.8378	69.4   
12 	200   	44.932 	60.1	12.1249	69.4   
13 	200   	30.808 	49.3	14.302 	69.4   
14 	200   	42.99  	63.1	10.6391	69.4   
15 	200   	45.776 	62.2	9.99633	69.4   
16 	200   	46.636 	64.3	13.5719	69.4   
17 	200   	50.156 	70  	11.7051	70     
18 	200   	48.184 	66.7	12.1521	70     
19 	200   	52.252 	64  	9.96543	70     
20 	200   	50.978 	67.9	10.068 	70     
21 	200   	52.586 	64.6	8.40293	70     
22 	200   	55.85  	66.1	7.12613	70     
2

Process ForkPoolWorker-923:
Process ForkPoolWorker-924:
Process ForkPoolWorker-922:
Process ForkPoolWorker-921:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._tar

KeyboardInterrupt: 

In [ ]:
#Trial 34 finished with value: 59.495698924731414 and parameters: {'train_enemies': [7, 8], 'initial_range': 1, 'pop_size': 50, 'n_offspring': 3, 'tournsize_parent': 10, 'cxpb': 0.30000000000000004, 'cx_type': 'blend', 'alpha': 0.9, 'mutpb': 0.8, 'mut_type': 'normal', 'sigma': 2.5, 'indpb': 'random', 'tau': 0.8, 'tournsize_survival': 7, 'ngen': 30, 'verbose': 1}

In [10]:
res_pd = pd.DataFrame(hof_tuning).sort_values('max_value',ascending=False).reset_index()

In [12]:
res_pd.to_csv("v3_res.csv")

In [ ]:
"""
params = {'initial_range':,
          'pop_size':,
          'n_offspring':,
          'tournsize_parent':,
          'cxpb':,
          'cx_type':,
          'alpha':,
          'mutpb':,
          'mut_type':,
          'sigma':,
          'indpb':,
          'tau':,
          'tournsize_survival':,
          'ngen':,
          'verbose':}
""""